# Agent

## Import libraries

In [ ]:
import os

import bs4
from typing import Annotated, Sequence, List
from typing_extensions import TypedDict
from langchain import hub
from langchain_openai import AzureChatOpenAI
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate

# RAG 
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# SQL DB
import sqlite3
import requests
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import AgentExecutor, create_sql_agent
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool

# Tools
from langchain.agents import create_openai_tools_agent
from langchain.tools import tool

# Langgraph
from langgraph.graph import StateGraph, END, START

from dotenv import load_dotenv

load_dotenv(override=True)

## Instantiate LLM

In [ ]:
llm = AzureChatOpenAI(
    azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_version = os.getenv('AZURE_OPENAI_API_VERSION'),
    api_key = os.getenv('AZURE_OPENAI_API_KEY'),
    deployment_name = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
)
# llm.invoke('Hey')

embeddings = AzureOpenAIEmbeddings(
    azure_deployment = os.getenv('AZURE_OPENAI_EMBEDDING'),
    api_version = os.getenv('AZURE_OPENAI_API_VERSION'),
    api_key = os.getenv('AZURE_OPENAI_EMBEDDING_API_KEY')
)
# embeddings.embed_query('test')

## Create Agents

### Agent State

In [ ]:
class AgentState(TypedDict):
    input: str
    output: str
    decision: str
    messages: Annotated[Sequence[BaseMessage], add_messages]

### RAG agent

In [ ]:
# VectorStore
vector_store = InMemoryVectorStore(embeddings)

# Load amd chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=('post-content', 'post-title','post-header')
        )
    ),
)

# Docs
docs = loader.load()

# Text Splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index Chunk
_ = vector_store.add_documents(documents=all_splits)

In [ ]:
# Option A

def format_docs(docs: List[Document]):
    return "\n\n".join(doc.page_content for doc in docs)

def rag_agent(state: AgentState):
    global llm
    rag_agent_llm = llm

    retriever = vector_store.as_retriever()

    system_prompt = """
        You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
        If you do not' know the answer, just say that you don't know. Use three sentences maximum and keep the answer consice.
        Question:{question}
        Context:{context}
        Answer:
    """
    prompt = ChatPromptTemplate([
        ('system', system_prompt + "Context: {context}"),
        ("human", "{question}")
    ])

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | rag_agent_llm
        | StrOutputParser()
    )

    # chain_multimodal_rag = (
    #    {
    #      'context': retriever | RunnableLambda(ingestion.get_image_description)
    #      'question': RunnablePassthrough()
    #    }
    #    | RunnableLambdba(ingestion.mutlimodel_prompt)
    #    | llm
    #    | StrOutputParser() 
    #response = rag_chain.invoke({'question': state['input']})
    response = rag_chain.invoke(state['input'])
    return {'output': response, 'input': state['input']}

### SQL Agent

In [ ]:
# SQL database

def get_engine_for_chinook_db():
    """Pull sql file, populate in-memory database, and create engine."""
    url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql"
    response = requests.get(url)
    sql_script = response.text

    connection = sqlite3.connect(":memory:", check_same_thread=False)
    connection.executescript(sql_script)
    return create_engine(
        "sqlite://",
        creator=lambda: connection,
        poolclass=StaticPool,
        connect_args={"check_same_thread": False},
    )


engine = get_engine_for_chinook_db()

db = SQLDatabase(engine)
sql_toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# sql_toolkit.get_tools()

In [ ]:
question =  "Which country's customers spent the most?"

In [ ]:
# Option A
from langchain.agents import AgentExecutor, create_react_agent
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm=llm, tools=sql_toolkit.get_tools(), prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=sql_toolkit.get_tools())

agent_executor.invoke({"input": question})

In [ ]:
# Option B
system_prompt_sql = """
    You are a helpful AI assistant expert in querying SQL Database to find answers to user's question about customers. If you can't find the answer, say ' I am unable to find the answer.'
"""
sql_agent = create_sql_agent(
    llm=llm,
    agent_type='openai-tools',
    toolkit=sql_toolkit,
    agent_exectutor_kwargs=dict(handle_parsing_errors=True)
)
prompt_sql = ChatPromptTemplate([
    ("system", system_prompt_sql),
    ("human", "{question}")
])

sql_agent.invoke(prompt_sql.format(question=question))

In [ ]:
def sql_agent(state):
    global llm
    sql_llm = llm

    engine = get_engine_for_chinook_db()

    db = SQLDatabase(engine)
    sql_toolkit = SQLDatabaseToolkit(db=db, llm=sql_llm)

    sql_agent = create_sql_agent(
        llm=llm,
        agent_type='openai-tools',
        toolkit=sql_toolkit,
        agent_exectutor_kwargs=dict(handle_parsing_errors=True)
    )
    prompt_sql = ChatPromptTemplate([
        ("system", system_prompt_sql),
        ("human", "{question}")
    ])
    
    response = sql_agent.invoke(prompt_sql.format(question=state['input']))

    return {'output': response['output'], 'input': state['input']}
        

### Tools

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
@tool
def buy():
    """Buys stuff"""
    return f'Bought stuff'

@tool
def sell():
    """Sells stuff"""
    return f'Sold 10 apples stuff'

# Tool agent
def tool_agent(state: AgentState):
    global llm
    tool_llm = llm

    prompt = ChatPromptTemplate([
        ("system", "You are an agent that ... "),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad")
    ])

    tools = [buy, sell]
    tool_agent = create_openai_tools_agent(
        tools=tools,
        llm=tool_llm,
        prompt=prompt
    )

    agent_executor = AgentExecutor(agent=tool_agent, tools=tools)

    response = agent_executor.invoke({'input': state['input']})

    return {'output':response, 'input': state['input']}

In [ ]:
from langchain_core.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper
os.environ["GOOGLE_CSE_ID"] = os.environ.get("GOOGLE_CSE_ID") 
os.environ["GOOGLE_API_KEY"] = os.environ.get("GOOGLE_API_KEY")
# Initialize Google Search tool
search = GoogleSearchAPIWrapper()
search_tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

### Rool Tools

In [ ]:
from typing import Literal
def route_tools(state: AgentState) -> Literal['rag','tool','sql',END]:
    if state['decision']=='rag':
        return "rag"
        
    if state['decision']=='tool':
        return "tool"

    if state['decision']=='sql':
        return 'sql'
        
    #last_message = state["messages"][-1]

    #if hasattr(last_message, "tool_calls") and last_message.tool_calls:
    #    tool_name = last_message.tool_calls[0]["name"]
    #    return tool_name
        
    return END

### Chat-Agent

In [ ]:
def chat_agent(state):
    global llm
    chat_llm = llm

    prompt = ChatPromptTemplate([
        ('system','You are agent who decised whether use rag, tool (for transaction, sell or buy) or sql agent. Only answer with rag, sql, or tool words'),
        ('human', '{input}')
        ]
    )
    chain = prompt | chat_llm

    response = chain.invoke({'input': state['input']})

    decision = response.content.strip().lower()

    return {'decision':decision}

## Langgraph

In [ ]:
def create_graph():
    workflow = StateGraph(AgentState)
    workflow.add_node('chat_agent', chat_agent)
    workflow.add_node('rag_agent', rag_agent)
    workflow.add_node('tool_agent', tool_agent)
    workflow.add_node('sql_agent', sql_agent)
    workflow.add_conditional_edges(
        "chat_agent",
        route_tools,
         {
            "rag": "rag_agent",
            "tool": "tool_agent",
            "sql": "sql_agent",
            END: END
        }
    )

    workflow.set_entry_point('chat_agent')
    workflow.add_edge('rag_agent',END)
    workflow.add_edge('tool_agent',END)
    workflow.add_edge('sql_agent',END)

    workflow = workflow.compile()

    return workflow
    

In [ ]:
from IPython.display import Image, display, Markdown, HTML
graph = create_graph()
display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

In [ ]:
import uuid  
thread_config = {"configurable": {"thread_id": str(uuid.uuid4())}} 
for event in graph.stream({'input': 'Tell me about agentic AI'}, config=thread_config, stream_mode=['updates']):
    print(event)

In [ ]:
import uuid  
thread_config = {"configurable": {"thread_id": str(uuid.uuid4())}} 
for event in graph.stream({'input': "Which country's customers spent the most?"}, config=thread_config, stream_mode=['updates']):
    print(event)

In [ ]:
import uuid  
thread_config = {"configurable": {"thread_id": str(uuid.uuid4())}} 
for event in graph.stream({'input': "Sell everything"}, config=thread_config, stream_mode=['updates']):
    print(event)